# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

In [77]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [78]:
import datetime

# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [79]:
def get_completion(prompt, model = llm_model, temperature = 0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )
    return response.choices[0].message.content

In [80]:
get_completion("What is 1+1?")

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

In [81]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [82]:
style = """American English \
in a calm and respectful tone
"""

prompt = f"""
Translate the text that is delimited by triple backticks \
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)


Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [83]:
response = get_completion(prompt)
print(response)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie. Unfortunately, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?


## Chat API : LangChain

Let's try how we can do the same using LangChain.

### Model

In [84]:
from langchain.chat_models import ChatOpenAI

In [85]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature 0.0
TEMPERATURE = 0.0

chat_model = ChatOpenAI(temperature = TEMPERATURE, model = llm_model)

### Prompt

In [86]:
template_string = """
Translate the text that is delimited by triple backticks \
into a style that is {style}. \
text: ````{text}```
"""

In [87]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [88]:
prompt_template.messages[0]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='\nTranslate the text that is delimited by triple backticks into a style that is {style}. text: ````{text}```\n'))

In [89]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='\nTranslate the text that is delimited by triple backticks into a style that is {style}. text: ````{text}```\n')

In [90]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [91]:
customer_style = """American English \
in a calm and respectful tone
"""

In [92]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

#### Two ways to format the prompts with our values

##### 1. Using Format Messages method
---

In [93]:
customer_messages = prompt_template.format_messages(
    style = customer_style,
    text = customer_email)

In [31]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [32]:
print(customer_messages[0])

content="\nTranslate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ````\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


##### 2. Using Invoke method
---

In [42]:
prompt_value = prompt_template.invoke({"style": customer_style, "text": customer_email})

In [43]:
print(type(prompt_value))
print(type(prompt_value.messages))
print(type(prompt_value.messages[0]))

<class 'langchain_core.prompt_values.ChatPromptValue'>
<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


**format_messages** method directly returns messages as a response.  
Whereas, **invoke** method returns **ChatPromptValue** as a response

#### Two ways to get a response from the chat model

##### 1. passing formatted message to the model
---

In [48]:
response = chat(customer_messages)
print(type(response))
print(response)

<class 'langchain_core.messages.ai.AIMessage'>
content='I am quite upset that my blender lid came off and made a mess of my kitchen walls with smoothie. Unfortunately, the warranty does not cover the cost of cleaning up my kitchen. I would greatly appreciate your assistance at this time, my friend.' response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 92, 'total_tokens': 142}, 'model_name': 'gpt-3.5-turbo-0301', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}


##### 2. Calling the invoke method and passing **prompt_value**
---

In [47]:
response = chat.invoke(prompt_value)
print(type(response))
print(response)

<class 'langchain_core.messages.ai.AIMessage'>
content='I am quite upset that my blender lid came off and made a mess of my kitchen walls with smoothie. Unfortunately, the warranty does not cover the cost of cleaning up my kitchen. I would greatly appreciate your assistance at this time, my friend.' response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 92, 'total_tokens': 142}, 'model_name': 'gpt-3.5-turbo-0301', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}


In [49]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [50]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [51]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)


Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ````Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [52]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, matey! I be sorry to inform ye that the warranty don't be coverin' the cost o' cleanin' yer galley 'cause 'tis yer own fault fer misusin' yer blender by forgettin' to put the lid on afore ye started it. Aye, tough luck! Farewell, me hearty!


**Conclusion**: __Why use Prompt Templates__:
- Prompt templates allows you to use useful prompts repeatedly.
- As the prompt gets lengthy, instead of rewriting the whole prompt again and again, templates allow you to change only the necessary parts without handling the whole prompt.

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [53]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [54]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [55]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [56]:
messages = prompt_template.format_messages(text = customer_review)
chat = ChatOpenAI(model = llm_model)
response = chat(messages)
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [58]:
type(response.content) # Not an actual dictionary

str

### Parse the LLM output string into a Python dictionary

In [59]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [60]:
gift_schema = ResponseSchema(name = "gift",
                             description = "Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name = "delivery_days",
                                      description = "How many days did it take for the product \
                                                    to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name = "price_value",
                                    description = "Extract any sentences about the value or price,\
                                                    and output them as a comma separated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [61]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [66]:
# stores the parsers instructions into a variable
format_instructions = output_parser.get_format_instructions()

In [67]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days did it take for the product                                                     to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price,                                                    and output them as a comma separated Python list.
}
```


Now, instead of writing the formatting information in the prompt, we pass out **format_instructions** received from **output_parser**

In [68]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [70]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [71]:
response = chat(messages)

In [72]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [73]:
output_dict = output_parser.parse(response.content)

In [74]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [75]:
type(output_dict)

dict

In [76]:
output_dict.get('delivery_days')

2